In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import keras_metrics as km
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import sys
import keras
from keras.models import load_model
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
np.set_printoptions(threshold=sys.maxsize)



Using TensorFlow backend.
/home/sharanya/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [10]:
x_train0 = np.load("trainx0.npy")
x_train1 = np.load("trainx1.npy")
x_train2 = np.load("trainx2.npy")

In [11]:
x_train0 = x_train0.reshape(1076, 300,300, 3).astype('float16') / 255
x_train1 = x_train1.reshape(1072, 300,300, 3).astype('float16') / 255
x_train2 = x_train0.reshape(1076, 300,300, 3).astype('float16') / 255

In [12]:
x_test0 = x_train0[:269]
y_test0 = np.zeros(269)

x_val0 = x_train0[269:430]
y_val0 = np.zeros(161)

x_train0 = x_train0[430:]
y_train0 = np.zeros(646)

In [13]:
x_test1 = x_train1[:269]
y_test1 = np.ones(269)

x_val1 = x_train1[269:430]
y_val1 = np.ones(161)

x_train1 = x_train1[430:]
y_train1 = np.ones(642)

In [14]:
x_test2 = x_train2[:269]
y_test2 = np.ones(269) * 2

x_val2 = x_train2[269:430]
y_val2 = np.ones(161) * 2

x_train2 = x_train2[430:]
y_train2 = np.ones(646) * 2

In [15]:
x_vali = np.append(x_val0, x_val1, axis=0)
x_val = np.append(x_vali, x_val2, axis=0)

x_testi = np.append(x_test0, x_test1, axis=0)
x_test = np.append(x_testi, x_test2, axis=0)

x_traini = np.append(x_train0, x_train1, axis=0)
x_train = np.append(x_traini, x_train2, axis=0)


y_vali = np.append(y_val0, y_val1, axis=0)
y_val = np.append(y_vali, y_val2, axis=0)

y_testi = np.append(y_test0, y_test1, axis=0)
y_test = np.append(y_testi, y_test2, axis=0)

y_traini = np.append(y_train0, y_train1, axis=0)
y_train = np.append(y_traini, y_train2, axis=0)

In [16]:
print(x_val.shape)
print(x_test.shape)
print(x_train.shape)

print(y_val.shape)
print(y_test.shape)
print(y_train.shape)

(483, 300, 300, 3)
(807, 300, 300, 3)
(1934, 300, 300, 3)
(483,)
(807,)
(1934,)


In [17]:
encoder = LabelEncoder()
encoder.fit(y_val)
encoded_Y = encoder.transform(y_val)
y_val_enc = np_utils.to_categorical(encoded_Y)

In [18]:
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
y_test_enc = np_utils.to_categorical(encoded_Y)

In [19]:
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
y_train_enc = np_utils.to_categorical(encoded_Y)

In [6]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, 300, 300)
else:
    input_shape = (300, 300, 3)

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(x_train,
    y_train_enc,
    epochs=3,
    batch_size=64,
    validation_data=(x_val,y_val_enc))

In [ ]:
model.save_weights('weights1.h5')

In [20]:
results = model.evaluate(x_test, y_test_enc, batch_size=128)
print('test loss, test acc:', results)

807/807 [==============================] - 108s 134ms/step
test loss, test acc: [0.009961007923919817, 0.997521698474884]


In [8]:
model.load_weights('weights1.h5')

In [21]:
# np.save('x_train.npy',x_train)
# np.save('x_test.npy',x_test)
# np.save('x_val.npy',x_val)

In [22]:
# np.save('y_train_enc.npy',y_train_enc)
# np.save('y_test_enc.npy',y_test_enc)
# np.save('y_val_enc.npy',y_val_enc)

In [23]:
y_pred = model.predict(x_test)

In [25]:
Y_pred = np.argmax(y_pred, axis=1)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, Y_pred))

[[268   1   0]
 [  1 268   0]
 [  0   0 269]]


In [28]:
target_names = ['covid', 'normal', 'pneumonia']
print(classification_report(y_test, Y_pred, target_names=target_names))


              precision    recall  f1-score   support

       covid       1.00      1.00      1.00       269
      normal       1.00      1.00      1.00       269
   pneumonia       1.00      1.00      1.00       269

    accuracy                           1.00       807
   macro avg       1.00      1.00      1.00       807
weighted avg       1.00      1.00      1.00       807

